In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.utils.data import DataLoader
import ResNetModel as models
from dataloader import LeukemiaLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def save_result(csv_path, predict_result, model_name):
    df = pd.read_csv(csv_path)
    new_df = pd.DataFrame()
    new_df['ID'] = df['Path']
    new_df["label"] = predict_result
    new_df.to_csv("./312554004_" + model_name + ".csv", index = False)
    
def test(model, model_name):
    model.eval()
    pred = []
    for i, (data, label) in enumerate(tqdm(test_dataloader)):
        with torch.no_grad():  
            data = data.to(device, dtype = torch.float)
            predicts = model(data)
            _, pred_num = torch.max(predicts, 1)    
            pred.extend(pred_num.detach().cpu().numpy().tolist())

    save_result(filename, pred, model_name)
    torch.cuda.empty_cache() 
    
if __name__ == '__main__':
#     summary(models.ResNet18().to(device), (3, 224, 224))
#     summary(models.ResNet50().to(device), (3, 512, 512))
#     summary(models.ResNet152().to(device), (3, 512, 512))
    
    all_model = ['ResNet18', 'ResNet50','ResNet152']
    
    for model_name in all_model:
        batch_size = 32
        if model_name == 'ResNet18':
            model = models.ResNet18()
            test_dataloader = DataLoader(LeukemiaLoader('', 'resnet_18'), batch_size = batch_size, shuffle = False)
            filename = 'resnet_18_test.csv'
        elif model_name == 'ResNet50':
            model = models.ResNet50()
            test_dataloader = DataLoader(LeukemiaLoader('', 'resnet_50'), batch_size = batch_size, shuffle = False)
            filename = 'resnet_50_test.csv'
        else:
            model = models.ResNet152()
            test_dataloader = DataLoader(LeukemiaLoader('', 'resnet_152'), batch_size = batch_size, shuffle = False)
            filename = 'resnet_152_test.csv'

        print(f'{model_name}:')
        model.load_state_dict(torch.load(f'./{model_name}.pt'))
        model = model.to(device)
        test(model, model_name)
        print('----------------------------')